In [1]:
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta
#import geopandas as gpd
#import matplotlib.pyplot as plt
#import descartes
import gurobipy as gp
from gurobipy import GRB

In [2]:
taxidata = pd.read_csv("yellow_tripdata_2010-01.csv")
taxidata['pickup_datetime'] = pd.to_datetime(taxidata['pickup_datetime'])
taxidata['dropoff_datetime'] = pd.to_datetime(taxidata['dropoff_datetime'])

date = datetime.date(2010, 1, 21)
data = taxidata.loc[(taxidata['pickup_datetime'].dt.date == date) & (taxidata['dropoff_datetime'].dt.date == date)]
data = data[['pickup_datetime', 'dropoff_datetime', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'fare_amount', 'surcharge', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount']]

n = 1000
data = data.sample(n).sort_values(by = 'pickup_datetime')
data.index = range(n)
data

C:\Users\holly\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,pickup_datetime,dropoff_datetime,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,2010-01-21 00:02:00,2010-01-21 00:10:00,1.17,-74.003597,40.723732,-73.989055,40.723298,6.1,0.5,0.5,0.00,0.0,7.10
1,2010-01-21 00:02:25,2010-01-21 00:10:58,1.30,-73.986082,40.759461,-73.969945,40.750948,6.5,0.5,0.5,0.00,0.0,7.50
2,2010-01-21 00:05:00,2010-01-21 00:14:00,4.23,-74.006418,40.748542,-73.966418,40.799348,10.9,0.5,0.5,0.00,0.0,11.90
3,2010-01-21 00:09:00,2010-01-21 00:41:00,8.02,-73.987745,40.719428,-73.973437,40.641503,23.7,0.5,0.5,4.84,0.0,29.54
4,2010-01-21 00:10:57,2010-01-21 00:21:47,3.00,-73.977206,40.784488,-73.944783,40.809992,9.7,0.5,0.5,0.00,0.0,10.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2010-01-21 23:43:00,2010-01-21 23:51:00,2.12,-73.968425,40.774598,-73.990317,40.762715,7.7,0.5,0.5,0.00,0.0,8.70
996,2010-01-21 23:44:01,2010-01-21 23:55:32,3.00,-73.981765,40.772536,-74.003030,40.736299,9.7,0.5,0.5,2.00,0.0,12.70
997,2010-01-21 23:44:34,2010-01-21 23:53:16,0.90,-73.984269,40.756973,-73.996623,40.752759,6.5,0.5,0.5,0.00,0.0,7.50
998,2010-01-21 23:48:03,2010-01-21 23:50:46,0.60,-73.985594,40.744932,-73.986199,40.739255,3.7,0.5,0.5,0.00,0.0,4.70


In [3]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 3956
    return c * r

In [28]:
mph = 20  # taxi speed between passengers
driving_cost = .6  # in dollars per mile

dist_between = np.zeros((n,n))   # Matrix of distances between dropoff/pickup of potential pairs of passengers
                                 # Note: =0 where i==j
possible = np.zeros((n,n))       # =1 if connection (i,j) is possible, =0 if impossible
profit_accept = np.zeros((n,n))  # Net profit from accepting passenger j (based on preceding passenger i)
                                 # Note: =0 if connection is impossible

for i in range(n):
    for j in range(n):        
        if (i == j): continue
        
        dist_between[i,j] = haversine(data.loc[i,'dropoff_longitude'], data.loc[i, 'dropoff_latitude'], data.loc[j, 'pickup_longitude'], data.loc[j, 'pickup_latitude'])
        
        time_between = timedelta(hours = dist_between[i,j] / mph)
        if (data.loc[i, 'dropoff_datetime'] + time_between > data.loc[j, 'pickup_datetime']): continue
        else: possible[i,j] = 1
        
        profit_accept[i,j] = data.loc[j, 'total_amount'] - driving_cost * (dist_between[i,j] + data.loc[j, 'trip_distance'])

In [35]:
def dist_between(i,j):
    if (i==j): return 0
    return haversine(data.loc[i,'dropoff_longitude'], data.loc[i, 'dropoff_latitude'], data.loc[j, 'pickup_longitude'], data.loc[j, 'pickup_latitude'])
def possible(i,j):
    if (i==j): return False
    time_between = timedelta(hours = dist_between(i,j) / mph)
    if (data.loc[i, 'dropoff_datetime'] + time_between > data.loc[j, 'pickup_datetime']): return False
    return True
def profit_accept(i,j):
    if (i==j | (not possible(i,j))): return 0
    return data.loc[j, 'total_amount'] - driving_cost * (dist_between(i,j) + data.loc[j, 'trip_distance'])

In [5]:
# locations = gpd.read_file('taxi_zones.shp')
# print(locations.crs)
# locations = locations.to_crs({'init': 'epsg:4326'})

# fig, ax = plt.subplots(figsize = (10,10))
# locations.plot(ax=ax)
# plt.show()

# def x_coord(i):
#     return locations.iloc[i,6].centroid.x
# def y_coord(i):
#     return locations.iloc[i,6].centroid.y